# Import libraries

In [13]:
from pathlib import Path
import subprocess

# point to your repo/data dirs
plink2 = "/home/mat2m10/code/mat2m10/chasm/plink2"
data_dir = Path("/home/mat2m10/code/mat2m10/chasm/data/1000G")

pgen = data_dir / "all_phase3.pgen"
pvar = data_dir / "all_phase3.pvar"
psam = data_dir / "phase3_corrected.psam"

clean_prefix = data_dir / "all_phase3_clean"


In [14]:
cmd = [
    plink2,
    "--pgen", str(pgen),
    "--pvar", str(pvar),
    "--psam", str(psam),
    "--set-all-var-ids", "@:#:$r:$a",
    "--new-id-max-allele-len", "200", "missing",
    "--snps-only", "just-acgt",
    "--max-alleles", "2",
    "--rm-dup", "exclude-mismatch",
    "--make-pgen",
    "--out", str(clean_prefix)
]

subprocess.run(cmd, check=True)

PLINK v2.0.0-a.7LM 64-bit Intel (1 Sep 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean.log.
Options in effect:
  --make-pgen
  --max-alleles 2
  --new-id-max-allele-len 200 missing
  --out /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean
  --pgen /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3.pgen
  --psam /home/mat2m10/code/mat2m10/chasm/data/1000G/phase3_corrected.psam
  --pvar /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3.pvar
  --rm-dup exclude-mismatch
  --set-all-var-ids @:#:$r:$a
  --snps-only just-acgt

Start time: Thu Sep 11 12:12:43 2025
15841 MiB RAM detected, ~14731 available; reserving 7920 MiB for main
workspace.
Using up to 28 threads (change this with --threads).
2504 samples (1270 females, 1233 males, 1 ambiguous; 2497 founders) loaded from
/home/mat2m10/code/mat2m10/chasm/data/1000G/phase3_corrected

CompletedProcess(args=['/home/mat2m10/code/mat2m10/chasm/plink2', '--pgen', '/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3.pgen', '--pvar', '/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3.pvar', '--psam', '/home/mat2m10/code/mat2m10/chasm/data/1000G/phase3_corrected.psam', '--set-all-var-ids', '@:#:$r:$a', '--new-id-max-allele-len', '200', 'missing', '--snps-only', 'just-acgt', '--max-alleles', '2', '--rm-dup', 'exclude-mismatch', '--make-pgen', '--out', '/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean'], returncode=0)

In [ ]:
out_dir = data_dir / "pca"
out_dir.mkdir(exist_ok=True)

# prune
prune_prefix = out_dir / "all_phase3_clean.prune"
cmd = [
    plink2, "--pfile", str(clean_prefix),
    "--indep-pairwise", "1000", "100", "0.05",
    "--threads", "8",
    "--out", str(prune_prefix)
]
subprocess.run(cmd, check=True)

# PCA (exact) — supports >100 PCs
pca_prefix = out_dir / "all_phase3_clean.PCA"
cmd = [
    plink2, "--pfile", str(clean_prefix),
    "--extract", str(prune_prefix) + ".prune.in",
    "--pca", "200",
    "--threads", "8",
    "--out", str(pca_prefix)
]
subprocess.run(cmd, check=True)



PLINK v2.0.0-a.7LM 64-bit Intel (1 Sep 2025)       cog-genomics.org/plink/2.0/
(C) 2005-2025 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/mat2m10/code/mat2m10/chasm/data/1000G/pca/all_phase3_clean.prune.log.
Options in effect:
  --indep-pairwise 1000 100 0.05
  --out /home/mat2m10/code/mat2m10/chasm/data/1000G/pca/all_phase3_clean.prune
  --pfile /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean
  --threads 8

Start time: Thu Sep 11 12:18:43 2025
15841 MiB RAM detected, ~15031 available; reserving 7920 MiB for main
workspace.
Using up to 8 compute threads.
2504 samples (1270 females, 1233 males, 1 ambiguous; 2497 founders) loaded from
/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean.psam.
81106350 variants loaded from
/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3_clean.pvar.
2 categorical phenotypes loaded.
Calculating alledone.
47%ndep-pairwise (7 compute threads): 0%

In [ ]:
import pandas as pd

eigvec = Path(str(pca_prefix) + ".eigenvec")
pcs_df = pd.read_csv(eigvec, delim_whitespace=True, header=None)
pcs_df.columns = ["FID", "IID"] + [f"PC{i}" for i in range(1, pcs_df.shape[1]-1)]


In [ ]:
pcs_df

NameError: name 'D' is not defined

In [11]:
import subprocess
from pathlib import Path
import pandas as pd

def ensure_unique_ids_and_clean(plink2, pgen, pvar, psam, out_prefix):
    cmd = [
        plink2, "--pgen", str(pgen), "--pvar", str(pvar), "--psam", str(psam),
        "--set-all-var-ids", "@:#:$r:$a",
        "--new-id-max-allele-len", "200", "missing",
        "--snps-only", "just-acgt",
        "--max-alleles", "2",
        "--rm-dup", "exclude-mismatch",
        "--make-pgen", "--out", str(out_prefix)
    ]
    subprocess.run(cmd, check=True, text=True, capture_output=True)

# usage inside pcs_from_pgen():
pgen = Path(D)/"all_phase3.pgen"; pvar = Path(D)/"all_phase3.pvar"; psam = Path(D)/"phase3_corrected.psam"
clean = Path(D)/"all_phase3_clean"
if not (clean.with_suffix(".pgen")).exists():
    ensure_unique_ids_and_clean(plink2, pgen, pvar, psam, clean)
# then run prune/PCA on 'all_phase3_clean' as the --pfile prefix


def sh(cmd):
    try:
        return subprocess.run(cmd, check=True, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print("COMMAND FAILED:\n", " ".join(cmd))
        print("\n--- STDERR ---\n", e.stderr)
        raise

def pcs_from_pgen(
    path_data="../../data/1000G",
    pfile_prefix="all_phase3",          # you have all_phase3.pgen/pvar/psam
    path_plink="../../",                 # dir containing plink2
    out_dir="../../data/1000G/pca",
    n_pcs=200, threads=8,
    longrange_bed=None,                 # e.g. "../../refs/longrangeLD.bed"
    indep_kb=1000, indep_step=100, indep_r2=0.05  # fairly aggressive to speed up
):
    path_data  = Path(path_data).resolve()
    out_dir    = Path(out_dir).resolve(); out_dir.mkdir(parents=True, exist_ok=True)
    plink2    = str((Path(path_plink).resolve() / "plink2").resolve())
    pfile_abs = str((path_data / pfile_prefix).resolve())

    # 1) (Optional) exclude long-range LD regions during pruning
    prune_tag = str((out_dir / f"{pfile_prefix}.prune").resolve())
    indep_cmd = [plink2, "--pfile", pfile_abs,
                 "--indep-pairwise", str(indep_kb), str(indep_step), str(indep_r2),
                 "--threads", str(threads), "--out", prune_tag]
    if longrange_bed:
        indep_cmd[2:2] = ["--exclude", "bed3", str(Path(longrange_bed).resolve())]
    sh(indep_cmd)

    prune_in = Path(prune_tag + ".prune.in")
    if not prune_in.exists() or sum(1 for _ in open(prune_in)) < max(1000, 3*n_pcs):
        # back off to a looser pruning if we were too aggressive
        prune_tag = str((out_dir / f"{pfile_prefix}.prune_loose").resolve())
        sh([plink2, "--pfile", pfile_abs,
            "--indep-pairwise", "1000", "50", "0.1",
            "--threads", str(threads), "--out", prune_tag])
        prune_in = Path(prune_tag + ".prune.in")

    # 2) PCA directly from PGEN + extract list (no intermediate BED paths!)
    pca_prefix = str((out_dir / f"{pfile_prefix}.PCA").resolve())
    sh([plink2, "--pfile", pfile_abs,
        "--extract", str(prune_in),
        "--pca", str(n_pcs), "approx",
        "--threads", str(threads),
        "--out", pca_prefix])

    # 3) Load to pandas
    eigvec = Path(pca_prefix + ".eigenvec")
    df = pd.read_csv(eigvec, delim_whitespace=True, header=None)
    df.columns = ["FID","IID"] + [f"PC{i}" for i in range(1, df.shape[1]-1)]
    # optional: save a convenient copy
    #df.to_parquet(Path(pca_prefix + ".parquet"), index=False)
    return df

pcs_df = pcs_from_pgen()
pcs_df.head()


COMMAND FAILED:
 /home/mat2m10/code/mat2m10/chasm/plink2 --pfile /home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3 --indep-pairwise 1000 100 0.05 --threads 8 --out /home/mat2m10/code/mat2m10/chasm/data/1000G/pca/all_phase3.prune

--- STDERR ---
 Error: --indep-pairwise requires unique variant IDs. (--set-all-var-ids and/or
--rm-dup may help.)



CalledProcessError: Command '['/home/mat2m10/code/mat2m10/chasm/plink2', '--pfile', '/home/mat2m10/code/mat2m10/chasm/data/1000G/all_phase3', '--indep-pairwise', '1000', '100', '0.05', '--threads', '8', '--out', '/home/mat2m10/code/mat2m10/chasm/data/1000G/pca/all_phase3.prune']' returned non-zero exit status 7.

In [4]:
pcs_df

NameError: name 'pcs_df' is not defined